In [47]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import emcee

In [48]:
df = pd.read_csv('VYCMa_processed.csv')
time = df['time'].values
mag = df['Mag'].values
good = time < 2019.5
time = time[good]
mag = mag[good]
from astropy.timeseries import LombScargle
ls = LombScargle(time, mag)
freq, power = ls.autopower()

In [49]:
def rednoise(frequency, alpha0, tau, gamma, alphaw):
    num = alpha0
    den = 1.0 + (2*np.pi*tau*frequency)**gamma
    return num/den + alphaw

def log_rednoise(frequency, alpha0, tau, gamma, alphaw):
    return np.log10(rednoise(frequency, alpha0, tau, gamma, alphaw))

from scipy.optimize import curve_fit

def fit_rednoise(time, mag):
    # Input: time and magnitude arrays
    
    # Step 1: Calculate the Lomb-Scargle Periodogram

    ls = LombScargle(time, mag)
    freq, power =ls.autopower()
    
    # Step 1.5: Calculate the log of the lomb-scargle periodogram
    log_power = np.log10(power) 
    
    # Step 2: Curve fit red noise function
    popt,pcov = curve_fit(log_rednoise, freq, log_power, p0 = (0.05, 0.1, 2, 0.01))
    
    alpha0, tau, gamma, alphaw = popt
   
    return alpha0, tau, gamma, alphaw

arg = fit_rednoise(time, mag)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log10
  import sys


In [50]:
def log_likelihood(theta, frequency, logpower):
    alpha0, tau, gamma, alphaw = theta
    model = np.log10(rednoise(frequency, alpha0, tau, gamma, alphaw))
    return -0.5 * np.sum((logpower - model) ** 2)

In [51]:
def log_prior(theta):
    alpha0, tau, gamma, alphaw = theta
    if (alphaw < alpha0) & (tau > 1/365):
        return 0.0
    return -np.inf

In [52]:
def log_probability(theta, frequency, logpower):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, frequency, logpower)

In [53]:
print (arg)
print(log_likelihood(arg, freq, np.log10(power)))

(0.026614760624789135, 0.3197256354612115, 2.6770428424755335, 0.00013430052057755989)
-15206.302171450443


In [54]:
import emcee

#arg = fit_rednoise(time, mag)

pos = arg + 1e-4 * np.random.randn(32, 4)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(freq, np.log10(power)))
sampler.run_mcmc(pos, 5000, progress=True);

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: Probability function returned NaN

In [ ]:
fig, axes = plt.subplots(4, figsize=(14, 7), sharex=True,dpi=300)
samples = sampler.get_chain()
labels = [r"$\alpha_0$", r"$\tau$", r"$\gamma$", r"$\alpha_w$"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
import corner
flat_samples = sampler.get_chain(discard=250, thin=15, flat=True)
fig = corner.corner(
    flat_samples, labels=labels,dpi=300)

In [ ]:
for i,val in enumerate(labels):
    print(i,val)

In [ ]:
for i, val in enumerate(labels):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    print(val,mcmc,f"{mcmc[1]:.3f}+{mcmc[2]-mcmc[1]:.3f}-{mcmc[1]-mcmc[0]:.3f}")